In [13]:
#import dataset
import pandas as pd
df = pd.read_excel("North Central Sales 6 Months GIT v2.xlsx", sheet_name="North Central")

#remove whitespace from column names for easy access
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('$', 'usd') for c in df.columns]
print(df.columns)

Index(['legacy_system_cd', 'legacy_division_cd', 'legacy_product_cd',
       'legacy_product_desc', 'core_item_flag', 'segment', 'PROD_CAT_1_NAME',
       'PROD_CAT_2_NAME', 'PROD_CAT_3_NAME', 'PROD_CAT_4_NAME',
       'legacy_vendor_cd', 'stocking_flag', 'LEGACY_CUSTOMER_CD',
       'saalfeld_customer_flag', 'national_acct_flag', 'ship-to_zip_code',
       'sales_channel', 'qty_6mos', 'cogs_6mos', 'Sales_6_mos', 'picks_6mos',
       'Margin_%', 'net_OH', 'net_OH_usd', 'pallet_quantity', 'item_poi_days',
       'DIOH'],
      dtype='object')


In [14]:
#Goal is to find the worst performing products
import numpy as np

#get the important products
imp_customers = df[df.national_acct_flag == "Y"]
important_products = imp_customers.legacy_product_cd.unique()
print(len(important_products))


products = df.legacy_product_cd.unique()
#print(len(products))

#Create a dictionary of products to sales
prod_to_sales = {}
for p in products:
    prod_to_sales[p] = 0

    
for p,s in zip(df.legacy_product_cd.values, df.Sales_6_mos.values):
    prod_to_sales[p] += s


#get the total value of sales
total_sales = np.nansum(list(prod_to_sales.values()))

#Create a dictionary of products to cogs
prod_to_cogs = {}
for p in products:
    prod_to_cogs[p] = 0

for p, c in zip(df.legacy_product_cd.values, df.cogs_6mos.values):
    prod_to_cogs[p] += c

#create a dictionary of products to profit = sales - cogs
prod_to_profit = {}
for p in products:
    prod_to_profit[p] = 0

for p in products:
    prod_to_profit[p] += prod_to_sales[p] - prod_to_cogs[p]

#get the total profit
total_profit = np.nansum(list(prod_to_profit.values()))

6547


In [15]:
#Create a dictionary that maps:
#Warehouse (Division) -> Unique products sold at division

warehouses = df.legacy_division_cd.unique()
warehouse_to_prod = {}

for w in warehouses:
    warehouse_to_prod[w] = []

for w, p in zip(df.legacy_division_cd.values, df.legacy_product_cd.values):
    for ware in warehouses:
        if w == ware and p not in warehouse_to_prod[w]:
            warehouse_to_prod[w].append(p)

#Print warehouse to number of individual different products stored            
for w in warehouses:
    print(w, len(warehouse_to_prod[w]))

19 2972
50 18
73 145
74 87
75 3543
77 1757
78 271
81 2513
82 2079
83 631
84 3596
85 1951
87 1534
89 3580
41 883


In [21]:
#Create a Dictionary of a warehouse, product cd pair with its net OH in units, net OH $ as values, item poi days & DIOH
#Example:
#Warehouse 19 houses product 10012415 and it has 4 units in on hand inventory, $81.2 of on hand inventory,
#302.430555555556 of average poi in days & 38.5263664820611 of DIOH
#The dictionary will look like this: wp_to_oh[19, 10012415] = [4.0, 81.2, 302.430555555556, 38.5263664820611]

wp_to_stats = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_stats[w,p] = [0,0,0,0]

for w, p, oh_usd, oh, poi, DIOH in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.net_OH_usd.values, df.net_OH.values,
                           df.item_poi_days.values, df.DIOH.values):
    if wp_to_stats[w,p] == [0,0,0,0]:
        wp_to_stats[w,p] = [oh, oh_usd, poi, DIOH]
        
print(wp_to_stats.items())

dict_items([((19, 10000116), [0.0, 0.0, 338.541668253581, 0.0]), ((19, 10012415), [4.0, 81.2, 302.430555555556, 38.5263664820611]), ((19, 10012430), [0.0, 0.0, 7.12561868531926, 0.0]), ((19, 10012515), [0.0, 0.0, 3786.95652173913, 0.0]), ((19, 10012628), [0.0, 0.0, 9999, 0.0]), ((19, 10022941), [0.0, 0.0, nan, nan]), ((19, 10023171), [0.0, 0.0, nan, nan]), ((19, 10023851), [0.0, 0.0, nan, nan]), ((19, 10024149), [0.0, 0.0, nan, nan]), ((19, 10024168), [0.0, 0.0, 53.6303631690521, 0.0]), ((19, 10024231), [0.0, 0.0, nan, nan]), ((19, 10024244), [0.0, 0.0, 5.91325706963115, 0.0]), ((19, 10024249), [0.0, 0.0, 17.177589856548, 0.0]), ((19, 10003866), [0.0, 0.0, nan, nan]), ((19, 10012422), [6.0, 172.8, 91.6842105263158, 29.6757919726982]), ((19, 10012630), [0.0, 0.0, 7.12561868531926, 0.0]), ((19, 10012878), [0.0, 0.0, 'No Venloc', 0.0]), ((19, 10023283), [3000.0, 276.66, 0.606286727015526, 11.0904610013328]), ((19, 10024004), [0.0, 0.0, nan, nan]), ((19, 10024183), [28000.0, 447.44, nan, n

In [ ]:
#Create a Dictionary of a warehouse, product cd pair with its item poi